In [ ]:
import logging

In [ ]:
logging.info('abc')

In [ ]:
import logging
import logging.handlers as handlers
import time

logger = logging.getLogger('my_app')
logger.setLevel(logging.INFO)

logHandler = handlers.TimedRotatingFileHandler('timed_app.log', when='M', interval=1)
logHandler.setLevel(logging.INFO)
logger.addHandler(logHandler)



def main():
    i = 0
    while True:
        i += 1
        time.sleep(1)
        logger.info(f"A Sample Log Statement {i}")

main()

- При запуске логирование производится в файл с именем из переменной **logname** (при отсутствии создается новый файл).
- После завершения интервала логирования создается еще один новый файл с именем **logname** + текст согласно паттерну **logHandler.suffix** (в паттерне в качестве времени берется время начала интервала). Файл содержит логи за только что завершенный интервал логирования (все переносится из файла **logname**).
- Сам файл **logname** очищается и далее в него в реальном времени пишутся логи за новый начатый интервал.
- Параметр **backupCount** определяет максимально число файлов логов, не работает без **logHandler.extMatch**.
- При прерывании логирования и запуске его с начала интервал смещается, данные в течение нового интерала дописываются в файл **logname**, и затем по завершении интервала также переносятся в файлы с логами за интервал, только в этих логах будет храниться непредсказуемое число логов и будет присутствовать разрыв.

In [ ]:
import re
import time
import datetime
import logging
import logging.handlers as handlers

logger = logging.getLogger('my_app')
logger.setLevel(logging.INFO)

border_time = datetime.time(second=0)

logHandler = handlers.TimedRotatingFileHandler('my_app.log', when='M', interval=1, delay=True, utc=True, atTime=border_time, backupCount=3)
logHandler.suffix = "%Y%m%d%H%M%S"

# Параметр extMatch необходим для того, чтобы очищать
# устаревшие логи, которые логгер должен найти по паттерну
# logname + suffix, в котором extMatch как раз задает
# паттерн для обнаружения суффикса.
logHandler.extMatch = re.compile(r"^\d{14}$")
logger.setLevel(logging.INFO)
logger.addHandler(logHandler)

def main():
    i = 0
    while True:
        i += 1
        time.sleep(1)
        logger.info(f"A Sample Log Statement: {i}")

try:
    main()
except KeyboardInterrupt:
    logHandler.close() # It is important to close handlers on application exit
    logging.shutdown()

In [ ]:
from datetime import datetime

datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
import time
import logging
import logging.handlers
from datetime import datetime
import os

logger = logging.getLogger('my_app')
logger.setLevel(logging.INFO)

# Dynamically generate the initial log filename with the current date
log_dir = "." # specify your log directory
current_date_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
initial_log_filename = os.path.join(log_dir, f"app_log_{current_date_str}.log")

def custom_namer(default_name: str = None):
    new_name = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return "_" + new_name + ".log"

logHandler = logging.handlers.TimedRotatingFileHandler(custom_namer, when="M", interval=1, utc=True, backupCount=3)
# Optional: define a custom namer to further adjust the rotated filenames if needed


# logHandler.suffix = "%Y-%m-%d %H:%M:%S.log" 


logHandler.namer = custom_namer
logger.addHandler(logHandler)

def main():
    i = 0
    while True:
        i += 1
        time.sleep(1)
        logger.info(f"A Sample Log Statement: {i}")

try:
    main()
except KeyboardInterrupt:
    logHandler.close() # It is important to close handlers on application exit
    logging.shutdown()

# Логирование разных уровней в разные файлы

In [ ]:
import logging
import logging.handlers as handlers
import time

logger = logging.getLogger('my_app')
logger.setLevel(logging.INFO)

## Here we define our formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

logHandler = handlers.TimedRotatingFileHandler('normal.log', when='M', interval=1, backupCount=0)
logHandler.setLevel(logging.INFO)
logHandler.setFormatter(formatter)

errorLogHandler = handlers.RotatingFileHandler('error.log', maxBytes=5000, backupCount=0)
errorLogHandler.setLevel(logging.ERROR)
errorLogHandler.setFormatter(formatter)

logger.addHandler(logHandler)
logger.addHandler(errorLogHandler)

def main():
    while True:
        time.sleep(1)
        logger.info("A Sample Log Statement")
        logger.error("An error log statement")

main()